# Simulate EEG source(s)

In this example, we explain how to simulate one or multiple sources to produce EEG measurements. The resulting `EEGSimulationSolution` can then be compared with the results obtained through evaluations of an `EEGForwardSolution`.

## Source definition

For this example we will simulate a single source. The process to add more is the same as you would only have to add `EEGSource` objects to the list.

In [1]:
from shamo import EEGSource

sources = [EEGSource((6, 6, 6), (1, 0, 0), 1e-6)]

An `EEGSource` is defined by its coordinates [mm], its orientation and its dipole moment [Am].

## Model creation

The model we create is almost the same as the one in [this example](1_model_creation.ipynb) so refer to it for more information. The only deferring step is the addition of the source(s).

This is achieved by using the `add_sources()` method. This method must be called only once per tissue that you want to add source(s) in. 

> **Note:**  
> The `add_sources()` method is destructive meaning it does not keep previous information such as previous source(s) or anisotropy. This is why we create a new model for the simulation.

In [2]:
import numpy as np
from shamo import FEModel, MeshConfig

sim_model = FEModel("model", 
                    "./derivatives/3_eeg_simulation")

# Generate mesh
labels = np.ones((11, 11, 11), dtype=np.uint8)
labels[3:-3, 3:-3, 3:-3] = 2
labels[2:-5, 2:-5, 2:-5] = 3
tissues = ["a", "b", "c"]
affine = np.diag([1, 1, 1, 1])
mesh_config = MeshConfig(facet_distance=1.5, cell_size=1)
sim_model.mesh_from_labels(labels, tissues, affine, 
                           mesh_config=mesh_config)

# Add source(s)
sim_model.add_sources(sources, "b", lc=1e-4)
sim_model.save()

# Add sensors
coordinates = [(x, y, z) for x in (0, 10) for y in (0, 10) 
               for z in (0, 10)]
sensors = {chr(ord("A") + i): c for i, c in enumerate(coordinates)}
sim_model.add_sensors(sensors, "a")

# Add anisotropy
field = np.ones((2, 2, 2)) * 0.1
field[1, :, :] = 1
field[:, 1, :] = 1
field[:, :, 1] = 1
affine = np.diag([10, 10, 10, 1])
sim_model.add_anisotropy_from_array(field, affine, "b",
                                fill_value=1e-8, formula="<b>")

# Save
sim_model.save()
print("Model created.")

Model created.


/usr/lib/python3/dist-packages/numpy/ctypeslib.py:531: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


If you open the resulting model in Gmsh, you can see that six points have been added to the model defining three pairs corresponding to the space axis.

## Problem definition

We first have to define a problem to solve. In this case, we create a `EEGSimulationProblem` which will generate a `EEGSimulationSolution`. This step is all the same as the [EEG forward problem](2_eeg_forward.ipynb) but one line.

In [3]:
from shamo import EEGSimulationProblem

problem = EEGSimulationProblem()

# Set the electical conductivities
sigmas = {"a": 1.0, "b": 0.5, "c": 0.25}
problem.set_electrical_conductivities(sigmas, {"b": "b_anisotropy"})

# Set the reference
problem.set_reference("A")

# Add markers
problem.add_markers(["C", "G"])

{'regions_of_interest': [],
 'markers': ['C', 'G'],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': 'A',
 'sources': []}

No need to add a region of interest here but we must specify the source(s) we want to simulate.

In [4]:
problem.add_sources(sources)

{'regions_of_interest': [],
 'markers': ['C', 'G'],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': 'b_anisotropy'},
  'c': {'value': 0.25, 'anisotropy': ''}},
 'reference': 'A',
 'sources': [{'coordinates': (0.006, 0.006, 0.006),
   'orientation': (1, 0, 0),
   'value': 1e-06,
   'unit_orientation': (1.0, 0.0, 0.0)}]}

## Problem resolution

Now that the problem is fully defined, we can solve it and look at the resulting recordings.

In [5]:
simulation = problem.solve("solution", 
                           "./derivatives/3_eeg_simulation",
                           sim_model)
print(simulation.recordings)

{'B': 0.0018330451402722991, 'D': 0.004812119135756493, 'E': -0.0060166481270252925, 'F': -0.007019520881107605, 'H': -0.009858387831023302}


Such a simulation not only provides sensors recordings but also three additional files:
- `v_skin.pos` contains the electrical potential [V] at every point of the skin.
- `v.pos` contains the electrical potential [V] in every element of the volume.
- `j.pos` contains the current density [Am$^{-2}$] in every element of the volume.